In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4089907a2318453af6fc2252e4da7284c4a4a98af4784e047e8ea9cdaaf83e7b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
import random
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql import Window

# Inicialize a sessão do Spark
spark = SparkSession.builder \
    .appName("ExemploDataFrameSpark") \
    .getOrCreate()

In [4]:
# Defina o número mínimo de registros desejados
num_registros = 10000

# Gere os dados aleatórios
data = []
for i in range(num_registros):
    data.append((
        i,  # ID
        random.randint(1, 4),             # Cluster1
        random.randint(1, 10),            # Cluster2
        round(random.uniform(150.00, 5000.00), 2),  # Valor
        round(random.uniform(0.20, 0.40), 2)      # Desconto
    ))

# Crie o DataFrame
columns = ["ID", "Cluster1", "Cluster2", "Valor", "Desconto"]
df = spark.createDataFrame(data, columns)

In [173]:
# Parametros

meta_valor = 5000000
meta_qtd = 3000
desconto_maximo = 0.7
MULTIPLICADOR = 2

In [69]:
def aplica_desconto(valor, desconto):
    return f.round(f.col(valor) * (1 - f.col(desconto)), 2)

In [68]:
window = Window.orderBy('Cluster1', 'Cluster2', f.desc('Valor')).rangeBetween(Window.unboundedPreceding, 0)

In [241]:
df1 = df.sort('Cluster1', 'Cluster2', f.desc('Valor'))\
        .withColumn('valorDescontado', aplica_desconto('Valor', 'Desconto'))\
        .withColumn('multiplicador', f.lit(MULTIPLICADOR))\
        .withColumn('desconto_majorado', f.col('Desconto') * f.col("multiplicador"))\
        .withColumn('valorDescontadoAjustado', aplica_desconto('Valor', 'desconto_majorado'))\
        .withColumn('somaAcumulada', f.round(f.sum('valorDescontado').over(window), 2))\
        .withColumn('qtdAcumulada', f.round(f.count('*').over(window), 2))\
        .withColumn('multiplicador_final', f.when( (f.col('somaAcumulada') <=  meta_valor) | (f.col('qtdAcumulada') <= meta_qtd), f.col('multiplicador'))\
                                         .otherwise(f.lit(1))
                   )\
        .withColumn('descontoAplicado', f.col('Desconto') * f.col('multiplicador_final'))\
        .withColumn('valorDescontadoAplicado', aplica_desconto('Valor', 'descontoAplicado'))\
        .withColumn('regraDesconto', f.when( (f.col('somaAcumulada') <=  meta_valor) | (f.col('qtdAcumulada') <= meta_qtd), f.lit(1))\
                                      .otherwise(f.lit(2))
                   )

In [242]:
df1.show()

+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+----------------+-----------------------+-------------+
|  ID|Cluster1|Cluster2|  Valor|Desconto|valorDescontado|multiplicador|desconto_majorado|valorDescontadoAjustado|somaAcumulada|qtdAcumulada|multiplicador_final|descontoAplicado|valorDescontadoAplicado|regraDesconto|
+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+----------------+-----------------------+-------------+
|7692|       1|       1|4973.22|    0.32|        3381.79|            2|             0.64|                1790.36|      3381.79|           1|                  2|            0.64|                1790.36|            1|
|1388|       1|       1|4970.31|    0.36|         3181.0|            2|             0.72|                1391.69|      6562.79|         

In [114]:
desconto_global = 1 - (df1.groupBy().sum('valorDescontadoAplicado').collect()[0][0] / df1.groupBy().sum('Valor').collect()[0][0])

In [115]:
desconto_global

0.3920539498876615

In [231]:
desconto_maximo_g1 = ( df1.where('regraDesconto = 1').groupBy().sum('Valor').collect()[0][0] * desconto_maximo )
desconto_maximo_g2 = ( df1.where('regraDesconto <> 1').groupBy().sum('Valor').collect()[0][0] * desconto_maximo )
desconto_utilizado_g1 = ( df1.where('regraDesconto = 1').groupBy().sum('Valor').collect()[0][0] - df1.where('regraDesconto = 1').groupBy().sum('valorDescontadoAplicado').collect()[0][0] )
desconto_utilizado_g2 = ( df1.where('regraDesconto <> 1').groupBy().sum('Valor').collect()[0][0] - df1.where('regraDesconto <> 1').groupBy().sum('valorDescontadoAplicado').collect()[0][0] )

In [232]:
# Valor de Desconto Restante (Potencial de Desconto Regra2 +/- Desconto Restante/Excedente da Regra1 )

desconto_disponivel = desconto_maximo_g1 + ( desconto_maximo_g2 - desconto_utilizado_g1 )

In [234]:
# Multiplicador que levará à utilização de 100% do desconto disponivel é obtido da razao entre o disponivel e o já utilizado no grupo

if desconto_global <= desconto_maximo:
    multiplicador_ajuste = desconto_disponivel / desconto_utilizado_g2
else:
    multiplicador_ajuste = 1

In [235]:
df2 = df1.withColumn('multiplicador_final', f.when(f.col('regraDesconto') == 1, f.col('multiplicador_final'))\
                                             .otherwise(f.lit(multiplicador_ajuste))
                     )\
         .withColumn('descontoAplicado', f.col('Desconto') * f.col('multiplicador_final'))\
         .withColumn('valorDescontadoAplicado', aplica_desconto('Valor', 'descontoAplicado'))

In [236]:
df2.show()

+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+----------------+-----------------------+-------------+
|  ID|Cluster1|Cluster2|  Valor|Desconto|valorDescontado|multiplicador|desconto_majorado|valorDescontadoAjustado|somaAcumulada|qtdAcumulada|multiplicador_final|descontoAplicado|valorDescontadoAplicado|regraDesconto|
+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+----------------+-----------------------+-------------+
|7692|       1|       1|4973.22|    0.32|        3381.79|            2|             0.64|                1790.36|      3381.79|           1|                2.0|            0.64|                1790.36|            1|
|1388|       1|       1|4970.31|    0.36|         3181.0|            2|             0.72|                1391.69|      6562.79|         

In [237]:
df2.where('regraDesconto <> 1').show()

+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+------------------+-----------------------+-------------+
|  ID|Cluster1|Cluster2|  Valor|Desconto|valorDescontado|multiplicador|desconto_majorado|valorDescontadoAjustado|somaAcumulada|qtdAcumulada|multiplicador_final|  descontoAplicado|valorDescontadoAplicado|regraDesconto|
+----+--------+--------+-------+--------+---------------+-------------+-----------------+-----------------------+-------------+------------+-------------------+------------------+-----------------------+-------------+
|5277|       2|       2| 347.72|    0.27|         253.84|            2|             0.54|                 159.95|   5507064.36|        3001| 2.4769870998595267|0.6687865169620723|                 115.17|            2|
|7935|       2|       2|  325.3|    0.34|          214.7|            2|             0.68|                  104.1|   5507279.06| 

In [238]:
# Desconto Global
1- (df2.groupBy().sum('valorDescontadoAplicado').collect()[0][0] / df2.groupBy().sum('Valor').collect()[0][0])

0.7000000382205231

In [239]:
1- (df2.where('regraDesconto <> 1').groupBy().sum('valorDescontadoAplicado').collect()[0][0] / df2.where('regraDesconto <> 1').groupBy().sum('Valor').collect()[0][0])

0.7421890532307414

In [240]:
1- (df2.where('regraDesconto = 1').groupBy().sum('valorDescontadoAplicado').collect()[0][0] / df2.where('regraDesconto = 1').groupBy().sum('Valor').collect()[0][0])

0.6034840256981681